In [1]:
import pandas
import pickle
from collections import defaultdict
from girvan_newman import GNDataset, GNModel, GNBetweenessGraph, GNModularityGraph

In [2]:
DUMMY_Graph = {
    "A": ["B", "C"],
    "B": ["A", "C", "D"],
    "C": ["A", "B"],
    "D": ["B", "E", "F", "G"],
    "E": ["D", "F"],
    "F": ["D", "E", "G"],
    "G": ["D", "F"]
}

In [3]:
# Betweeness with given root node
gn_betweeness_graph = GNBetweenessGraph(DUMMY_Graph)
print("Given 'E' as root, Betweenes")
for l in sorted(gn_betweeness_graph.betweeness('E'), key=lambda x: x[0]):
    print(l[0], l[1])


# Overall betweeness
vertices = list(DUMMY_Graph.keys())
edges_to_betweeness = dict()
for v1, v2s in DUMMY_Graph.items():
    for v2 in v2s:
        edges_to_betweeness[(min(v1, v2), max(v1, v2))] = 0
for v in vertices:
    betweenesses = gn_betweeness_graph.betweeness(v)
    for edge, betweeness in betweenesses:
        edges_to_betweeness[edge] += betweeness / 2

print("\nOverall betweeness")
for k, v in edges_to_betweeness.items():
    print(k, v)

Given 'E' as root, Betweenes
('A', 'B') 1.0
('B', 'C') 1.0
('B', 'D') 3.0
('D', 'E') 4.5
('D', 'G') 0.5
('E', 'F') 1.5
('F', 'G') 0.5

Overall betweeness
('A', 'B') 5.0
('A', 'C') 1.0
('B', 'C') 5.0
('B', 'D') 12.0
('D', 'E') 4.5
('D', 'F') 4.0
('D', 'G') 4.5
('E', 'F') 1.5
('F', 'G') 1.5


In [4]:
# Find highest betweeness edge with spark
# Map: calculation betweeness given one vertice as root node
# Reduce: Find the edge with highest betweeness
gn_model = GNModel(GNDataset(DUMMY_Graph))
gn_model.highest_betweeness_edges()

[('B', 'D')]

In [5]:
# Modularity
gn_modularity_graph = GNModularityGraph(DUMMY_Graph)

edges = set()
for v1, v2s in DUMMY_Graph.items():
    for v2 in v2s:
        edges.add((min(v1, v2), max(v1, v2)))

for v1, v2 in edges:
    DUMMY_Graph[v1].remove(v2)
    DUMMY_Graph[v2].remove(v1)
    print("Remove ({}, {}), modularity: {:.4f}".format(v1, v2, gn_modularity_graph.modularity(DUMMY_Graph)))
    DUMMY_Graph[v1].append(v2)
    DUMMY_Graph[v2].append(v1)

Remove (A, B), modularity: 0.0000
Remove (B, C), modularity: 0.0000
Remove (A, C), modularity: 0.0000
Remove (F, G), modularity: 0.0000
Remove (B, D), modularity: 0.3642
Remove (D, E), modularity: 0.0000
Remove (E, F), modularity: 0.0000
Remove (D, F), modularity: 0.0000
Remove (D, G), modularity: 0.0000


In [6]:
# Find Communities
gn_model = GNModel(GNDataset(DUMMY_Graph))
gn_model.communities

INFO:girvan_newman.model:Iter:  1, Modularity: 0.3642
INFO:girvan_newman.model:Iter:  2, NOT highest modularity
INFO:girvan_newman.model:FINISH


[['A', 'B', 'C'], ['D', 'E', 'F', 'G']]

In [7]:
# Find Communities in a much larger graph
larger_graph = pickle.load(open('./large_graph.pkl', 'rb'))
gn_model = GNModel(GNDataset(larger_graph), 0.001)
communities = gn_model.communities
print('--------------Communities-----------------')
for c in communities:
    print(c)

INFO:girvan_newman.model:Iter:  1, Modularity: 0.0785
INFO:girvan_newman.model:Iter:  2, NOT highest modularity
INFO:girvan_newman.model:Iter:  3, NOT highest modularity
INFO:girvan_newman.model:Iter:  4, Modularity: 0.5247
INFO:girvan_newman.model:Iter:  5, NOT highest modularity
INFO:girvan_newman.model:Iter:  6, NOT highest modularity
INFO:girvan_newman.model:Iter:  7, NOT highest modularity
INFO:girvan_newman.model:Iter:  8, NOT highest modularity
INFO:girvan_newman.model:Iter:  9, NOT highest modularity
INFO:girvan_newman.model:Iter: 10, NOT highest modularity
INFO:girvan_newman.model:Iter: 11, NOT highest modularity
INFO:girvan_newman.model:Iter: 12, Modularity: 0.6249
INFO:girvan_newman.model:Iter: 13, Modularity: 0.6491
INFO:girvan_newman.model:Iter: 14, NOT highest modularity
INFO:girvan_newman.model:Iter: 15, NOT highest modularity
INFO:girvan_newman.model:Iter: 16, Modularity: 0.6527
INFO:girvan_newman.model:Iter: 17, Modularity: 0.6823
INFO:girvan_newman.model:Iter: 18, NOT